In [1]:
pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
cp kaggle.json ~/.kaggle/

In [8]:
! chmod 600~/ .kaggle/kaggle.json

chmod: invalid mode: ‘600~/’
Try 'chmod --help' for more information.


In [9]:
! kaggle datasets download paramaggarwal/fashion-product-images-small

 99% 559M/565M [00:04<00:00, 161MB/s]
100% 565M/565M [00:04<00:00, 132MB/s]


In [10]:
! unzip -q 'fashion-product-images-small.zip'

In [11]:
#training the model 
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm

In [12]:
#splitting the training set and creating training directory for it
import os
import random
import shutil
from sklearn.model_selection import train_test_split

# Set the paths
data_dir = '/content/images'
train_dir = './train1'
test_dir = './test1'

# Create the train and test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Set the test size
test_size = 0.95

# Get the list of filenames
filenames = os.listdir(data_dir)

# Get the train and test filenames
train_filenames, test_filenames = train_test_split(filenames, test_size=test_size, random_state=42)
print(len(train_filenames))
print(len(test_filenames))

# Copy the files to the split directories
for filename in train_filenames:
    shutil.copy(os.path.join(data_dir, filename), os.path.join(train_dir, filename))

for filename in test_filenames:
    shutil.copy(os.path.join(data_dir, filename), os.path.join(test_dir, filename))


2222
42219


In [13]:
#importing resnet model for extracting feature
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

94765736/94765736 [==============================] - 0s 0us/step


In [14]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

print(model.summary())    # model sumary that how resnet model will extract 2048 features from every image used for training in the dataset

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [15]:
# function to extract the feature from the image and m=normalizing the result
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result

In [16]:
import pickle

In [17]:
#training the model and storing the feature in a pickle file for further use.
filenames = []


for file in os.listdir('train1'):
    filenames.append(os.path.join('train1',file))

feature_list = []

for file in tqdm(filenames):
    feature_list.append(extract_features(file,model))

pickle.dump(feature_list,open('embeddings.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))

  0%|          | 0/2222 [00:00<?, ?it/s]

1/1 [==============================] - 9s 9s/step


  0%|          | 1/2222 [00:09<5:42:21,  9.25s/it]

1/1 [==============================] - 0s 34ms/step


  0%|          | 3/2222 [00:09<1:31:12,  2.47s/it]

...

100%|█████████▉| 2219/2222 [03:10<00:00, 11.97it/s]

1/1 [==============================] - 0s 29ms/step


100%|█████████▉| 2221/2222 [03:10<00:00, 11.92it/s]

1/1 [==============================] - 0s 24ms/step


100%|██████████| 2222/2222 [03:10<00:00, 11.64it/s]


In [18]:
import os    #get the directory where the pickle file is geting saved
print(os.getcwd())


/content


In [19]:
import os     #navigate to the directory
os.chdir('/content')


In [20]:
#Testing the neural  model
import pickle
import tensorflow
import numpy as np
from numpy.linalg import norm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
import cv2



In [21]:
#loading the trained feature above stored in pickle file
feature_list = np.array(pickle.load(open('embeddings.pkl','rb')))
filenames = pickle.load(open('filenames.pkl','rb'))

In [22]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

In [23]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

In [25]:
#testing the model on an image that is a simple shirt ,first the features frm this imae is 
#extracted and used to match with the already trained images to give result
img = image.load_img('faith-yarn-Wr0TpKqf26s-unsplash.jpg',target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

1/1 [==============================] - 1s 838ms/step


In [26]:
#nearest neihbour algorithm is used tor eturn the images who are approximaetly matching the features of the test image
neighbors = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')
neighbors.fit(feature_list)

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=6)

In [27]:
#indicies of around 5 images that approximaetly matches the test image is returned 
distances,indices = neighbors.kneighbors([normalized_result])

print(indices)

[[ 794 1880  889  533  129  585]]


In [40]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 68.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [63]:
%%writefile main1.py
import streamlit as st


Writing main1.py


In [64]:
!pip install  pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
from pyngrok import ngrok

In [67]:
import streamlit as st
import os
from PIL import Image
import numpy as np
import pickle
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm

In [68]:
feature_list = np.array(pickle.load(open('embeddings.pkl','rb')))
filenames = pickle.load(open('filenames.pkl','rb'))

In [69]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

In [70]:
st.title("Fashon with AI")
def save_uploaded_file(uploaded_file):
    try:
        with open(os.path.join('uploads',uploaded_file.name),'wb') as f:
            f.write(uploaded_file.getbuffer())
        return 1
    except:
        return 0


In [71]:
def feature_extraction(img_path,model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result

In [72]:
def recommend(features,feature_list):
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)

    distances, indices = neighbors.kneighbors([features])

    return indices

In [73]:
#import streamlit as st
uploaded_file = st.file_uploader("Choose an image")
if uploaded_file is not None:
    if save_uploaded_file(uploaded_file):
        # display the file
        display_image = Image.open(uploaded_file)
        st.image(display_image)
        # feature extract
        features = feature_extraction(os.path.join("uploads",uploaded_file.name),model)
        #st.text(features)
        # recommendention
        indices = recommend(features,feature_list)
        # show
        col1,col2,col3,col4,col5 = st.beta_columns(5)

        with col1:
            st.image(filenames[indices[0][0]])
        with col2:
            st.image(filenames[indices[0][1]])
        with col3:
            st.image(filenames[indices[0][2]])
        with col4:
            st.image(filenames[indices[0][3]])
        with col5:
            st.image(filenames[indices[0][4]])
    else:
        st.header("Some error occured in file upload")

In [74]:
!streamlit run main1.py & npx localtunnel --port 8501

npx: installed 22 in 1.899s


your url is: https://funny-poets-shout.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.222.61.7:8501

  Stopping...
^C


In [77]:
!zip -r fashiom.zip /content/fashiom.ipynb



	zip warning: name not matched: /content/fashiom.ipynb

zip error: Nothing to do! (try: zip -r fashiom.zip . -i /content/fashiom.ipynb)
